<a href="https://colab.research.google.com/github/tak-creator/giganci/blob/main/finetunning_GPT2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

In [2]:
from transformers import AutoTokenizer, GPT2LMHeadModel, pipeline, Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset

In [3]:
model_name = "flax-community/papuGaPT2"
# Tokenizer wspólny dla obu modeli
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Model przed treningiem
model_orig = GPT2LMHeadModel.from_pretrained(model_name)
model_orig.resize_token_embeddings(len(tokenizer))

# Model, który będziemy trenować
model_tuned = GPT2LMHeadModel.from_pretrained(model_name)
model_tuned.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/888k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/547k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/864 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

Embedding(50257, 768)

In [4]:
generator_orig = pipeline('text-generation', model=model_orig, tokenizer=tokenizer)
prompt = "gra: Stray \n opis: "
wynik = generator_orig(prompt, max_length=100, num_return_sequences=1)
print(wynik[0]['generated_text'])

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


gra: Stray 
 opis:  puzzle, które nie posiadają elementów, które są niebezpieczne dla najmłodszych. Zestaw nie zawiera elementów niebezpiecznych dla dzieci. Puzzle do układania, w zestawie... Pełen opis produktu 'Puzzle do układania - Krowa i konik morski - CAPTUR' »
Opis: Puzzle do układania, w zestawie z figurką z serii "Psi Patrol" oraz z serii "Psi detektyw". Wiek 4+, opakowanie zawiera 6 elementów.
Puzzle w kształcie krabów. Wymiary obrazka: 32 x 32 cm. Wiek: 4+ Zawartość: 12 elementów.
Puzzle w kształcie krabów. Wymiary obrazka: 32 x 32 cm. Wiek: 4+ Zawartość: 12 elementów.
Puzzle w kształcie krabów. Wymiary obrazka: 32 x 32 cm. Wiek: 4+ Zawartość: 12 elementów.
Puzzle w kształcie krabów. Wymiary obrazka: 32 x 32 cm. Wiek: 4+ Zawartość: 12 elementów.
Puzzle w kształcie krabów. Wymiary obrazka: 32 x 32 cm. Wiek: 4+ Zawartość: 12 elementów.
Puzzle w kształcie krabów. Wymiary obrazka: 32 x 32 cm. Wiek: 4+ Zawartość: 12 elementów.
Puzzle w kształcie krabów. Wymiary obrazka: 32 x 32 c

In [5]:
generator_orig = pipeline("text-generation", model=model_orig, tokenizer=tokenizer)

prompt = "Gra: Hogwards Legacy\nOpis:"
wynik = generator_orig(
    prompt,
    max_new_tokens=100,
    num_return_sequences=1,
    temperature=0.9)
print("Tekst PRZED treningiem:")
print(wynik[0]["generated_text"])

Device set to use cpu


Tekst PRZED treningiem:
Gra: Hogwards Legacy
Opis: Witaj, czarodzieju. Poszłam w góry, jak przystało na księżniczkę, gdzie po jednej wyprawie, poznałam wielu nowych ludzi. Byłam na spotkaniu z Czarnoksiężnikiem Twardowskim, do którego należał mój ojciec, który także z kolei był złoczyńcą. Miałam wielu przyjaciół w całej okolicy, a w szczególności moją matkę, która mieszkała z rodzicami w pobliżu zamku. Po kilku dniach wyszłam z zamku. Po wyjściu z zamku, mój przyjaciel, powiedział, że mnie


trzeba dołączyc plik TXT "gry"

In [6]:
with open("gry.txt", "r", encoding="utf-8") as f:
    text = f.read()

texty = [blok.strip() for blok in text.split("===") if blok.strip()]

dataframe = pd.DataFrame({"text": texty})
dataset = Dataset.from_pandas(dataframe)

In [7]:
tokenizer.pad_token = tokenizer.eos_token #token do dopełniania -> token do sekfencji
def tokenize(batch): #batch partia danych
    tokens = tokenizer(batch["text"], padding=True, truncation=True, # padding = dopełnić sekwencje do najdłuższej lub przyciąć max i min 512!
    max_length=512)
    tokens["labels"] = tokens["input_ids"].copy() # nowy klucz dodaje labels = kopia lista tokenów wejściowych
    return tokens # zwraca słownik :)
tokenized_dataset = dataset.map(tokenize, batched=True) # nowy zbiór danych z txt na tokeny i maski do modelu

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [8]:
#Fine-tuning modelu
training_args = TrainingArguments(
    output_dir="./papuGaGPT2-finetuned",
    num_train_epochs=10, # 2 epoki
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    logging_steps=10,
    save_steps=1000,
    save_total_limit=1,
    report_to="none"
    )
trainer = Trainer(
    model=model_tuned,
    args=training_args,
    train_dataset=tokenized_dataset,
    )
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,2.053000
20,0.651300
30,0.223300


TrainOutput(global_step=30, training_loss=0.9758966604868571, metrics={'train_runtime': 147.0458, 'train_samples_per_second': 0.34, 'train_steps_per_second': 0.204, 'total_flos': 1964793600000.0, 'train_loss': 0.9758966604868571, 'epoch': 10.0})

In [9]:
#Generowanie opisu nowej gry po treningu

generator_tuned = pipeline("text-generation", model=model_tuned, tokenizer=tokenizer)

prompt = "Gra: Hogward Legacy\nOpis:"
wynik = generator_tuned(
    prompt,
    max_new_tokens=100,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.8,
    num_beams=3
    )
print(" Tekst PO treningu:")
print(wynik[0]["generated_text"])

Device set to use cpu


 Tekst PO treningu:
Gra: Hogward Legacy
Opis: Hogwarts Legacy przenosi gracza do XIX-wiecznej szkoły magii w świecie Harry’ego Pottera. Jako uczeń tworzysz własnego bohatera, uczysz się zaklęć, odkrywasz tajemnice i walczysz z czarną magią. Gra zachwyca szczegółowym światem i możliwością eksploracji poza murami Hogwartu.
